<a href="https://colab.research.google.com/github/DevThiagoSc/Analise_Venda_Imoveis/blob/main/Converter_coordenadas_endereco.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from geopy.geocoders import Nominatim

In [ ]:
geolocator = Nominatim(user_agent="priceAnalysis", timeout=15)

In [ ]:
df = pd.read_csv('kc_house_data.csv')

In [ ]:
df.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


In [ ]:
def get_address(lat, lon):
    location = geolocator.reverse((lat, lon))
    return location.address

In [ ]:
def get_neighborhood(location):
    address = location.raw['address']
    neighborhood = address.get('neighbourhood', '')
    city = address.get('city', '')
    return pd.Series([neighborhood, city])

    return pd.Series([neighborhood, city])

In [ ]:
df['localization'] = df.apply(lambda x: str(x['lat']) + ';' + str(x['long']), axis=1)

In [ ]:
df.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,localization
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,...,1180,0,1955,0,98178,47.5112,-122.257,1340,5650,47.5112;-122.257
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,...,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639,47.721;-122.319
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,...,770,0,1933,0,98028,47.7379,-122.233,2720,8062,47.7379;-122.233
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,...,1050,910,1965,0,98136,47.5208,-122.393,1360,5000,47.5208;-122.393
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,...,1680,0,1987,0,98074,47.6168,-122.045,1800,7503,47.6168;-122.045


In [ ]:
neighborhood_city = df.localization.apply(lambda x: geolocator.reverse(x)).apply(get_neighborhood)

In [ ]:
neighborhood_city.sample(10)

,0,1
19180,,Seattle
14541,,
18968,Sand Point,Seattle
4094,,
17578,Morgan Junction,Seattle
5850,,
10897,,Seattle
613,,
1726,Twin Lakes,
14705,South Park,Seattle


In [ ]:
neighborhood_city.to_csv('neighborhood_city.csv', index=False)

In [ ]:
df_2 = pd.read_csv('neighborhood_city.csv')

In [ ]:
df_2.columns = ['neighborhood', 'city']

In [ ]:
df_2.isnull().sum()

neighborhood    13849
city            13196
dtype: int64

In [ ]:
df_2.isna().sum()

neighborhood    13849
city            13196
dtype: int64

In [ ]:
df_2.neighborhood.nunique()

367

In [ ]:
df = pd.concat([df, df_2], axis=1)
df

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,neighborhood,city
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,...,0,1955,0,98178,47.5112,-122.257,1340,5650,Rainier Beach,Seattle
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,...,400,1951,1991,98125,47.7210,-122.319,1690,7639,NaN,Seattle
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,...,0,1933,0,98028,47.7379,-122.233,2720,8062,NaN,NaN
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,...,910,1965,0,98136,47.5208,-122.393,1360,5000,Fauntleroy,Seattle
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,...,0,1987,0,98074,47.6168,-122.045,1800,7503,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21608,263000018,20140521T000000,360000.0,3,2.50,1530,1131,3.0,0,0,...,0,2009,0,98103,47.6993,-122.346,1530,1509,Licton Springs,Seattle
21609,6600060120,20150223T000000,400000.0,4,2.50,2310,5813,2.0,0,0,...,0,2014,0,98146,47.5107,-122.362,1830,7200,NaN,Seattle
21610,1523300141,20140623T000000,402101.0,2,0.75,1020,1350,2.0,0,0,...,0,2009,0,98144,47.5944,-122.299,1020,2007,NaN,Seattle
21611,291310100,20150116T000000,400000.0,3,2.50,1600,2388,2.0,0,0,...,0,2004,0,98027,47.5345,-122.069,1410,1287,NaN,NaN


In [ ]:
houses_seattle = df.query('city == "Seattle"')
houses_seattle.shape

(7087, 23)

In [ ]:
houses_seattle.neighborhood.nunique()

76

In [ ]:
houses_seattle.neighborhood.unique()

array(['Rainier Beach', nan, 'Fauntleroy', 'Wedgwood', 'Sunset Hill',
       'Licton Springs', 'Bitter Lake', 'Whittier Heights', 'Ravenna',
       'Crown Hill', 'Laurelhurst', 'East Campus', 'NewHolly', 'Kenwood',
       'Morgan Junction', 'Lawton Park', 'Loyal Heights', 'Montlake',
       'Dunlap', 'Lakeridge', 'High Point', 'Van Asselt', 'Brighton',
       'Arbor Heights', 'Rainier Vista', 'University Park',
       'Highland Park', 'South Seattle', 'North Beach', 'Interbay',
       'Portage Bay', 'Hawthorne Hills', 'Alki', 'Meridian',
       'Arroyo Heights', 'Admiral', 'Canterbury', 'Westwood',
       'Carleton Park', 'Blue Ridge', 'Magnolia Village', 'View Ridge',
       'Windermere', 'Alicia Park', 'Youngstown', 'Edgewater Park',
       'Brighton Beach', 'Seward Park', 'Cedar Park', 'Tamill',
       'Victory Heights', 'Southeast Magnolia', 'Uptown', 'Little Saigon',
       'Beverly Park', 'West Campus', 'Lakewood', 'South Lake Union',
       'Columbia City', 'Columbia', 'Westlake

In [ ]:
houses_seattle.dropna(inplace=True)

/usr/local/lib/python3.9/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [ ]:
houses_seattle.neighborhood.value_counts()

Whittier Heights    157
Highland Park       139
Wedgwood            133
Licton Springs      123
Admiral             123
                   ... 
Seaview               5
Mount View            5
Greek Row             4
Highline              2
South Campus          1
Name: neighborhood, Length: 76, dtype: int64

In [ ]:
kc_houses_data_seattle = houses_seattle.query('neighborhood == "Whittier Heights" or neighborhood == "Highland Park" or neighborhood == "Wedgwood"')

In [ ]:
kc_houses_data_seattle.neighborhood.value_counts()

Whittier Heights    157
Highland Park       139
Wedgwood            133
Name: neighborhood, dtype: int64

In [ ]:
kc_houses_data_seattle.shape

(429, 23)

In [ ]:
kc_houses_data_seattle.to_csv('kc_houses_data_seattle.csv', index=False)